## NBA API

### Description
- Ingest data via nba_api
- reformat data
- push data to Supabase
- This notebook will is meant to explore the nba_api package and test push some data to Supabase
- Future python files will be designed to package up consistent data pulls from the api and updates to Supabase tables
- Once data is consistently landed, I will work on building some ML models and Neural Networks to add Data Science flair to the project
- Final step will be packaging it up in a simple web app to allow others to interact

### Import Packaages

In [93]:
## import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import nba_api
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.static import teams as static_teams
from nba_api.stats.static import players as static_players
from nba_api.stats.endpoints import leaguegamefinder
import time
import requests
import json
import seaborn as sns
import sys
import os
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()

True

### Test pull to grab recent ATL games

In [ ]:

atl = 1610612737  # Atlanta Hawks team_id
games = leaguegamefinder.LeagueGameFinder(team_id_nullable=atl).get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22025,1610612737,ATL,Atlanta Hawks,0022500223,2025-11-12,ATL @ SAC,W,240,133,...,0.818,7,38,45,42,14.0,6,19,20,33.0
1,22025,1610612737,ATL,Atlanta Hawks,0022500206,2025-11-10,ATL @ LAC,W,239,105,...,0.800,7,34,41,30,7.0,8,11,22,3.0
2,22025,1610612737,ATL,Atlanta Hawks,0022500185,2025-11-08,ATL vs. LAL,W,240,122,...,0.909,7,30,37,37,13.0,6,11,14,20.0
3,22025,1610612737,ATL,Atlanta Hawks,0022500030,2025-11-07,ATL vs. TOR,L,241,97,...,0.957,19,35,54,24,8.0,7,15,19,-12.0
4,22025,1610612737,ATL,Atlanta Hawks,0022500166,2025-11-04,ATL vs. ORL,W,240,127,...,0.810,6,28,34,23,11.0,6,18,24,15.0


In [6]:
games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

### Lets do some broader things to get data
- first use the static databases to get team and player IDs (thesse can be dim tables)
- then lets flush out some broader functionality with usefful endpoints

#### Grab team IDs

In [8]:
# All teams (list[dict])
teams_list = static_teams.get_teams()
teams_df = pd.DataFrame(teams_list).sort_values("full_name")
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
14,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
29,1610612766,Charlotte Hornets,CHA,Hornets,Charlotte,North Carolina,1988
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [59]:
def get_team_ids_df():
    teams_list = static_teams.get_teams()
    teams_df = pd.DataFrame(teams_list).sort_values("full_name")
    teams_df.rename(columns={'id': 'team_id'}, inplace=True)
    return teams_df

In [60]:
teams_df = get_team_ids_df()
teams_df.head()

,team_id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
14,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
29,1610612766,Charlotte Hornets,CHA,Hornets,Charlotte,North Carolina,1988
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


#### Create some helper dictionaries 

In [9]:
# Helper maps
TEAM_BY_ABBR = {t["abbreviation"]: t for t in teams_list}
TEAM_BY_ID   = {t["id"]: t for t in teams_list}

# Example lookups
TEAM_BY_ABBR["ATL"]["id"]         # -> 1610612737
TEAM_BY_ID[1610612737]["full_name"]  # -> "Atlanta Hawks"


'Atlanta Hawks'

In [18]:
def get_team_id():
    teams_list = static_teams.get_teams()
    TEAM_BY_ID   = {t["id"]: t for t in teams_list}
    return TEAM_BY_ID

In [20]:
team_id_dicts = get_team_id()

#### Grab Player IDs

In [14]:
# All players (current + historical)
players_list = static_players.get_players()
players_df = pd.DataFrame(players_list)
players_df.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [15]:
# Find by name (returns list; pick the first if unique)
static_players.find_players_by_full_name("LeBron James")

[{'id': 2544,
  'full_name': 'LeBron James',
  'first_name': 'LeBron',
  'last_name': 'James',
  'is_active': True}]

In [35]:
def get_player_df():
    players_list = static_players.get_players()
    players_df = pd.DataFrame(players_list)
    players_df.rename(columns={'id': 'player_id'}, inplace=True)
    return players_df

In [36]:
players_df = get_player_df()
players_df.head()

,player_id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


#### Lets look at the game logs endpoint

In [ ]:
## here is one call to get all atl games

atl = 1610612737  # Atlanta Hawks team_id
games = leaguegamefinder.LeagueGameFinder(team_id_nullable=atl).get_data_frames()[0]

games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22025,1610612737,ATL,Atlanta Hawks,0022500223,2025-11-12,ATL @ SAC,W,240,133,...,0.818,7,38,45,42,14.0,6,19,20,33.0
1,22025,1610612737,ATL,Atlanta Hawks,0022500206,2025-11-10,ATL @ LAC,W,239,105,...,0.800,7,34,41,30,7.0,8,11,22,3.0
2,22025,1610612737,ATL,Atlanta Hawks,0022500185,2025-11-08,ATL vs. LAL,W,240,122,...,0.909,7,30,37,37,13.0,6,11,14,20.0
3,22025,1610612737,ATL,Atlanta Hawks,0022500030,2025-11-07,ATL vs. TOR,L,241,97,...,0.957,19,35,54,24,8.0,7,15,19,-12.0
4,22025,1610612737,ATL,Atlanta Hawks,0022500166,2025-11-04,ATL vs. ORL,W,240,127,...,0.810,6,28,34,23,11.0,6,18,24,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806,21983,1610612737,ATL,Atlanta Hawks,0028300041,1983-11-04,ATL vs. CHI,W,240,103,...,0.692,19,27,46,31,14.0,13,18,27,NaN
3807,21983,1610612737,ATL,Atlanta Hawks,0028300027,1983-11-01,ATL vs. WAS,W,240,95,...,0.633,12,29,41,20,7.0,10,16,34,NaN
3808,21983,1610612737,ATL,Atlanta Hawks,0028300014,1983-10-29,ATL vs. DET,W,240,117,...,0.633,27,21,48,28,14.0,7,23,35,NaN
3809,21983,1610612737,ATL,Atlanta Hawks,0028300005,1983-10-28,ATL @ NJN,L,240,108,...,0.684,15,18,33,14,10.0,5,17,30,NaN


In [27]:
team_id_list = [t for t in teams_df['id']]

In [30]:
team_games = [] 
for t in teams_df['id']:
    games_df = leaguegamefinder.LeagueGameFinder(team_id_nullable=t).get_data_frames()[0]
    team_games.append(games_df)

team_games_df = pd.concat(team_games)

#### Make sure we got all the data

In [31]:
team_games_df['TEAM_NAME'].value_counts()

TEAM_NAME
Los Angeles Lakers                   4089
Boston Celtics                       4037
San Antonio Spurs                    3978
Utah Jazz                            3913
Chicago Bulls                        3883
Houston Rockets                      3880
Dallas Mavericks                     3877
Detroit Pistons                      3874
Phoenix Suns                         3868
Indiana Pacers                       3866
Portland Trail Blazers               3845
Cleveland Cavaliers                  3840
Philadelphia 76ers                   3840
Golden State Warriors                3839
Denver Nuggets                       3833
New York Knicks                      3829
Milwaukee Bucks                      3822
Atlanta Hawks                        3812
Sacramento Kings                     3547
Miami Heat                           3542
Orlando Magic                        3235
Minnesota Timberwolves               3205
Toronto Raptors                      2742
Los Angeles Clippers    

In [32]:
team_games_df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [81]:
# Step 1: rename columns to match Supabase schema
fact_team_games_df = team_games_df.rename(columns={
    "SEASON_ID": "season_id",
    "TEAM_ID": "team_id",
    "GAME_ID": "game_id",
    "GAME_DATE": "game_date",
    "MATCHUP": "matchup",
    "WL": "wl",
    "PTS": "pts",
    "FG_PCT": "fg_pct",
    "FG3_PCT": "fg3_pct",
    "FT_PCT": "ft_pct",
    "REB": "reb",
    "AST": "ast",
    "STL": "stl",
    "BLK": "blk",
    "TOV": "tov",
    "PLUS_MINUS": "plus_minus",
})

# Step 2: define the exact column order your Supabase table uses
supabase_col_order = [
    "game_id",
    "team_id",
    "game_date",
    "matchup",
    "wl",
    "pts",
    "plus_minus",
    "season_id",
    "fg_pct",
    "fg3_pct",
    "ft_pct",
    "reb",
    "ast",
    "stl",
    "blk",
    "tov",
]

# Step 3: restrict to those columns and reorder
fact_team_games_df = fact_team_games_df[supabase_col_order]

# Step 4: convert dates to ISO format (Supabase/Postgres friendly)
fact_team_games_df["game_date"] = (
    fact_team_games_df["game_date"].astype("datetime64[ns]").dt.date
)

fact_team_games_df.head()


,game_id,team_id,game_date,matchup,wl,pts,plus_minus,season_id,fg_pct,fg3_pct,ft_pct,reb,ast,stl,blk,tov
0,0022500227,1610612737,2025-11-13,ATL @ UTA,None,21,8.0,22025,0.667,0.571,0.500,7.0,5,2.0,0,2
1,0022500223,1610612737,2025-11-12,ATL @ SAC,W,133,33.0,22025,0.576,0.425,0.818,45.0,42,14.0,6,19
2,0022500206,1610612737,2025-11-10,ATL @ LAC,W,105,3.0,22025,0.452,0.382,0.800,41.0,30,7.0,8,11
3,0022500185,1610612737,2025-11-08,ATL vs. LAL,W,122,20.0,22025,0.516,0.410,0.909,37.0,37,13.0,6,11
4,0022500030,1610612737,2025-11-07,ATL vs. TOR,L,97,-12.0,22025,0.344,0.237,0.957,54.0,24,8.0,7,15


In [82]:
# Ensure game_date is a datetime type
fact_team_games_df["game_date"] = pd.to_datetime(fact_team_games_df["game_date"]).dt.date

# Define cutoff
cutoff_date = pd.to_datetime("2025-11-13").date()

# Filter rows
fact_team_games_df = fact_team_games_df[fact_team_games_df["game_date"] < cutoff_date]

fact_team_games_df.head()

,game_id,team_id,game_date,matchup,wl,pts,plus_minus,season_id,fg_pct,fg3_pct,ft_pct,reb,ast,stl,blk,tov
1,0022500223,1610612737,2025-11-12,ATL @ SAC,W,133,33.0,22025,0.576,0.425,0.818,45.0,42,14.0,6,19
2,0022500206,1610612737,2025-11-10,ATL @ LAC,W,105,3.0,22025,0.452,0.382,0.800,41.0,30,7.0,8,11
3,0022500185,1610612737,2025-11-08,ATL vs. LAL,W,122,20.0,22025,0.516,0.410,0.909,37.0,37,13.0,6,11
4,0022500030,1610612737,2025-11-07,ATL vs. TOR,L,97,-12.0,22025,0.344,0.237,0.957,54.0,24,8.0,7,15
5,0022500166,1610612737,2025-11-04,ATL vs. ORL,W,127,15.0,22025,0.556,0.433,0.810,34.0,23,11.0,6,18


In [83]:
# 1) Ensure datetime, then convert to ISO string
fact_team_games_df["game_date"] = (
    pd.to_datetime(fact_team_games_df["game_date"])
      .dt.strftime("%Y-%m-%d")   # now plain strings like "2025-11-12"
)

# 2) (Recommended) Replace NaN/NaT with None so JSON can handle them
fact_team_games_df = fact_team_games_df.replace({np.nan: None})

In [84]:
## drop duplicates in the raw data
fact_team_games_df = fact_team_games_df.drop_duplicates(
    subset=["game_id", "team_id"],
    keep="first"
).reset_index(drop=True)

In [89]:
fact_team_games_df.shape

(109412, 16)

#### Test write to Supabase

#### Connect to Supabase

In [43]:
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_SERVICE_KEY")  # or anon key for de

In [44]:
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [45]:
def upsert_in_chunks(table_name, records, chunk_size=500, conflict_cols=None):
    for i in range(0, len(records), chunk_size):
        chunk = records[i:i+chunk_size]
        query = supabase.table(table_name)
        if conflict_cols:
            query = query.upsert(chunk, on_conflict=",".join(conflict_cols))
        else:
            query = query.insert(chunk)
        resp = query.execute()
        # Optional: print or check resp
        print(f"{table_name}: upserted rows {i}–{i+len(chunk)-1}")

#### Convert dataframes to json

In [61]:
dim_teams_records = teams_df.to_dict(orient="records")

In [64]:
dim_players_records = players_df.to_dict(orient="records")

In [91]:
fact_team_games_records = fact_team_games_df.to_dict(orient="records")

In [63]:
# dim_teams
upsert_in_chunks("dim_teams", dim_teams_records, chunk_size=200, conflict_cols=["team_id"])


dim_teams: upserted rows 0–29


In [65]:
# dim_teams
upsert_in_chunks("dim_players", dim_players_records, chunk_size=200, conflict_cols=["player_id"])


dim_players: upserted rows 0–199
dim_players: upserted rows 200–399
dim_players: upserted rows 400–599
dim_players: upserted rows 600–799
dim_players: upserted rows 800–999
dim_players: upserted rows 1000–1199
dim_players: upserted rows 1200–1399
dim_players: upserted rows 1400–1599
dim_players: upserted rows 1600–1799
dim_players: upserted rows 1800–1999
dim_players: upserted rows 2000–2199
dim_players: upserted rows 2200–2399
dim_players: upserted rows 2400–2599
dim_players: upserted rows 2600–2799
dim_players: upserted rows 2800–2999
dim_players: upserted rows 3000–3199
dim_players: upserted rows 3200–3399
dim_players: upserted rows 3400–3599
dim_players: upserted rows 3600–3799
dim_players: upserted rows 3800–3999
dim_players: upserted rows 4000–4199
dim_players: upserted rows 4200–4399
dim_players: upserted rows 4400–4599
dim_players: upserted rows 4600–4799
dim_players: upserted rows 4800–4999
dim_players: upserted rows 5000–5102


In [92]:
# fact_team_games
upsert_in_chunks(
    "fact_team_games",
    fact_team_games_records,
    chunk_size=500,
    conflict_cols=["game_id", "team_id"],
)

fact_team_games: upserted rows 0–499
fact_team_games: upserted rows 500–999
fact_team_games: upserted rows 1000–1499
fact_team_games: upserted rows 1500–1999
fact_team_games: upserted rows 2000–2499
fact_team_games: upserted rows 2500–2999
fact_team_games: upserted rows 3000–3499
fact_team_games: upserted rows 3500–3999
fact_team_games: upserted rows 4000–4499
fact_team_games: upserted rows 4500–4999
fact_team_games: upserted rows 5000–5499
fact_team_games: upserted rows 5500–5999
fact_team_games: upserted rows 6000–6499
fact_team_games: upserted rows 6500–6999
fact_team_games: upserted rows 7000–7499
fact_team_games: upserted rows 7500–7999
fact_team_games: upserted rows 8000–8499
fact_team_games: upserted rows 8500–8999
fact_team_games: upserted rows 9000–9499
fact_team_games: upserted rows 9500–9999
fact_team_games: upserted rows 10000–10499
fact_team_games: upserted rows 10500–10999
fact_team_games: upserted rows 11000–11499
fact_team_games: upserted rows 11500–11999
fact_team_game